In [17]:
!pip install keras-tuner

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [0]:
fashion_mnist=keras.datasets.fashion_mnist

In [0]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [0]:
# we are scaling down the pixels of image between 0 to 1 as images are in grayscale
train_images=train_images/255.0
test_images=test_images/255.0

In [22]:
train_images[0].shape   #shape of 1st image of train_images

(28, 28)

In [0]:
#reshaping the images so that they become easier to train
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
#hp.choice is used to choose the different combinations of hyperparameters
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch  #Random Serach is used for tuning hyperparameters
from kerastuner.engine.hyperparameters import HyperParameters

In [26]:
#tuner_search is object
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [27]:
#3 epochs for each trial
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [0]:
model=tuner_search.get_best_models(num_models=1)[0]    #[0]to convert in form of list

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1179712   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,199,466
Trainable params: 1,199,466
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1549 - accuracy: 0.9425 - val_loss: 0.2529 - val_accuracy: 0.9140
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1183 - accuracy: 0.9561 - val_loss: 0.2657 - val_accuracy: 0.9140
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0874 - accuracy: 0.9677 - val_loss: 0.3218 - val_accuracy: 0.9103
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0622 - accuracy: 0.9771 - val_loss: 0.3313 - val_accuracy: 0.9145
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0495 - accuracy: 0.9820 - val_loss: 0.3771 - val_accuracy: 0.9143
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0360 - accuracy: 0.9866 - val_loss: 0.4241 - val_accuracy: 0.9108
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0304 - accuracy: 0.9894 - val_loss: 0.5058 - val_accuracy